In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import matplotlib.dates as mdates
import os, sys
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm
import pandas as pd


def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

r1 = MakeDataFromText("Temperature 50p5/RSSI_66212.csv")
t1 = MakeDataFromText("Temperature 50p5/tags_66212.csv")
r2 = MakeDataFromText("Temperature 50p8/RSSI_95451.csv")
t2 = MakeDataFromText("Temperature 50p8/tags_95451.csv")
r3 = MakeDataFromText("40p7/RSSI_56307.csv")
t3 = MakeDataFromText("40p7/tags_56307.csv")
r4 = MakeDataFromText("56p2 Data 1/RSSI_44986.csv")
t4 = MakeDataFromText("56p2 Data 1/tags_44986.csv")
r5 = MakeDataFromText("56p2 Data 2/RSSI_44926.csv")
t5 = MakeDataFromText("56p2 Data 2/tags_44926.csv")
t6 = MakeDataFromText("55p5/tags_69303.csv")
r6 = MakeDataFromText("55p5/RSSI_69303.csv")
t7 = MakeDataFromText("23p7/tags_13318.csv")
r7 = MakeDataFromText("23p7/RSSI_13318.csv")
t8 = MakeDataFromText("23p2/tags_60650.csv")
r8 = MakeDataFromText("23p2/RSSI_60650.csv")

temp23p2 = MakeDict([t8])
rssi23p2 = MakeDict([r8])
temp50p5 = MakeDict([t1])
rssi50p5 = MakeDict([r1])
temp50p8 = MakeDict([t2])
rssi50p8 = MakeDict([r2])
temp40p7 = MakeDict([t3])
rssi40p7 = MakeDict([r3])
temp56p2_1 = MakeDict([t4])
rssi56p2_1 = MakeDict([r4])
temp56p2_2 = MakeDict([t5])
rssi56p2_2 = MakeDict([r5])
temp55p5 = MakeDict([t6])
rssi55p5 = MakeDict([r6])
temp23p7 = MakeDict([t7])
rssi23p7 = MakeDict([r7])

In [ ]:
LABEL = ["23.2", "23.7", "40.7", "50.5", "50.8", "55.5", "56.2 #1", "56.2 #2"]
TEMP = [temp23p2, temp23p7, temp40p7, temp50p5, temp50p8, temp55p5, temp56p2_1, temp56p2_2]
RSSI = [rssi23p2, rssi23p7, rssi40p7, rssi50p5, rssi50p8, rssi55p5, rssi56p2_1, rssi56p2_2]
FITCOLOR = ["#ff0000", "#ff6f00", "#ff14d0", "#fa0505", "#f07e05", "#fa0505"]
RAWCOLOR = ["#0DA995", "#0024c2", "#0F78BC", "#008ec2", "#110063", "#008ec2"]
TAG = "3943"

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1, figsize=(12, 24), sharex=True)
axs = [ax1, ax2, ax3, ax4, ax5, ax6]

for u in range(6):
    for k,v in TEMP[u].items():
        if k[-4:]==TAG:
            st, sr = sorted( TEMP[u][k].items() ), sorted( RSSI[u][k].items() )
            dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

            # Making Standard Deviation and Error Bars
            means, rssi = [], []
            c1 = { i:[] for i in range(26) }
            for i in range(len(dt)): c1[dr[i]].append(dt[i])
            for k,v in c1.items():
                if len(v)>0 and k<100:
                    mean = sum(v)/len(v)
                    std = np.std(v)
                    rssi.append(k)
                    means.append(mean)
                    axs[u].scatter(k, mean, marker='o',color='b')
                    axs[u].errorbar(k, mean, yerr=std, color='b')
                    axs[u].text(k, mean+std, str(round(std,2)), fontsize=9, color="b", horizontalalignment='center', verticalalignment='bottom')
            
            # try:
            if True:
                # Making Best Fits, Linear & Quadratic
                model1 = np.poly1d( np.polyfit(rssi, means,  1) )
                model2 = np.poly1d( np.polyfit(rssi, means,  2) )

                x1, y1 = zip( *sorted( zip(rssi, model1(rssi)) ) )
                x2, y2 = zip( *sorted( zip(rssi, model2(rssi)) ) )

                strmodel1 = ", " + str(round(model1[1], 4))+"x + "+str(round(model1[0], 4))
                strmodel2 = ", " + str(round(model2[2], 4))+"x^2 + "+str(round(model2[1], 4))+"x + "+str(round(model2[0], 4))

                axs[u].plot(x1, y1, color=FITCOLOR[u], label="R2=" + str(round(r2_score(means, y1),3)) + strmodel1, linewidth=2.0)
            # except:
            #     pass
            # axs[u].plot(x2, y2, color=FITCOLOR[u], label="R2=" + str(round(r2_score(means, y2),3)) + strmodel2, linestyle='--', )


axs[0].set_xticks(range(27))
axs[1].set_xticks(range(27))
axs[2].set_xticks(range(27))
axs[3].set_xticks(range(27))
axs[4].set_xticks(range(27))
axs[5].set_xticks(range(27))
ax1.set_title(TAG + ", 23.2 C")
ax2.set_title(TAG + ", 23.7 C")
ax3.set_title(TAG + ", 40.7 C")
ax4.set_title(TAG + ", 50.5 C")
ax5.set_title(TAG + ", 50.8 C")
ax6.set_title(TAG + ", 55.5 C")
ax1.legend(); ax2.legend(); ax3.legend(); ax4.legend(); ax5.legend(); ax6.legend()
ax1.set_xlabel("RSSI (dBm)"); ax2.set_xlabel("RSSI (dBm)"); ax3.set_xlabel("RSSI (dBm)");
ax4.set_xlabel("RSSI (dBm)"); ax5.set_xlabel("RSSI (dBm)"); ax6.set_xlabel("RSSI (dBm)");
ax1.set_ylabel("Temperature (C)")
ax2.set_ylabel("Temperature (C)")
ax3.set_ylabel("Temperature (C)")
ax4.set_ylabel("Temperature (C)")
ax5.set_ylabel("Temperature (C)")
ax6.set_ylabel("Temperature (C)")
ax1.xaxis.set_tick_params(labelbottom=True)
ax2.xaxis.set_tick_params(labelbottom=True)
ax3.xaxis.set_tick_params(labelbottom=True)
ax4.xaxis.set_tick_params(labelbottom=True)
ax5.xaxis.set_tick_params(labelbottom=True)
ax6.xaxis.set_tick_params(labelbottom=True)
fig.subplots_adjust(hspace=0.32)